Required Imports:

In [1]:
import sys, os
from pathlib import Path
current_folder = globals()["_dh"][0]
vasp_path = os.path.join(str(Path(current_folder).parent),r"src\vasp")
print(vasp_path)
sys.path.append(vasp_path)  # add vasp directory to PATH
from vasp import VASP
from data_handler import DATA_HANDLER

c:\Users\nc298\Documents\repos\vasp\src\vasp


First define the:
* input file
* voxel size
* voxel space origin
* configure the voxelisation

In this example the voxel size is 0.25 m, the origin is at 0,0,0.
The vasp will calculate the mean for the RGB values and additionally calculate the point_count.
The output will than be stored in the outfile. Here each voxels center will be used as the X,Y,Z coordinate.
Additionally the output will be stored in a ply file containing a mesh of voxels.

In [2]:
# Point cloud to be voxelized
infiles = [r"C:\vasp_test_data\vasp_in.laz"]
voxel_size = 1
origin = [0,0,0]
outfile = r"C:\vasp_test_data\vasp_out.laz"
outfile_ply = r"C:\vasp_test_data\vasp_out.ply"

This is where all implemented attributes are calculated. For each function called, the result is stored as an attribute in the outfile. However, for geometric features, the covariance and eigenvalues must be run beforehand, but it is sufficient to call only geometric features. If only geometric features is called, the covariance matrix and eigenvalues are not stored in the outfile.

In [3]:
config = {
    "infiles":infiles,
    "voxel_size":voxel_size,
    "origin":origin,
    "attributes":{
        "red":"mean",
        "green":"mean",
        "blue":"mean"
        },
    "calculate":[   
        "big_int_index",
        "hash_index",
        "point_count",
        "point_density",
        "percentage_occupied",
        "covariance_matrix",
        "eigenvalues",
        "geometric_features",
        "center_of_gravity",
        "center_of_voxel",
        "corner_of_voxel"
        ],
    "return_at":"center_of_gravity"
    }

This is a common example of how to define a filter. 

In [4]:
vox_filter = {
    "filter_attribute":"point_count",
    "filter_value":5,
    "min_max_eq":"min"}

Open the data with the DATA_HANDLER. Also give information about what attributes to read.

In [5]:
data_handler = DATA_HANDLER(
    config["infiles"],
    config["attributes"]
    )
data_handler.load_las_files()

Calling load_las_files
C:\vasp_test_data\vasp_in.laz
Loading ... vasp_in.laz
Function 'load_las_files' executed in 0.0025 seconds


Initiate the VOXELIZER with all required parameters. Than pass the data from the data_handler to the vasp.

In [6]:
vasp = VASP(
    config["voxel_size"],
    config["origin"],
    config["attributes"],
    config["calculate"],
    config["return_at"]
    )

vasp.get_data_from_data_handler(
    data_handler
    )

Calling get_data_from_data_handler
Function 'get_data_from_data_handler' executed in 0.0000 seconds


Compute the requested attributes and statistics per voxel as defined in the config file. After computing all new attributes and statistics, reduce the data. Only one value per voxel remains for each attribute.

In [7]:

vasp.compute_requested_attributes()
vasp.compute_requested_statistics_per_attributes() #Currently faster, does only support min, mean, median, max, sum
# vasp.compute_requested_statistics_per_attributes_numpy() #Currently slower, does additionaly support mode
vasp.reduce_to_voxels()

Calling compute_requested_attributes
Calling compute_big_int_index
Calling voxelize
Function 'voxelize' executed in 0.0060 seconds
Function 'compute_big_int_index' executed in 0.0120 seconds
Calling compute_hash_index
Function 'compute_hash_index' executed in 0.0000 seconds
Calling compute_point_count
Function 'compute_point_count' executed in 0.0000 seconds
Calling compute_point_density
Function 'compute_point_density' executed in 0.0000 seconds
Calling compute_percentage_occupied
54.63 of the voxel space is occupied
Function 'compute_percentage_occupied' executed in 0.0000 seconds
Calling compute_covariance_matrix
Function 'compute_covariance_matrix' executed in 0.0529 seconds
Calling compute_eigenvalues
Function 'compute_eigenvalues' executed in 0.0430 seconds
Calling compute_geometric_features
Function 'compute_geometric_features' executed in 0.0000 seconds
Calling compute_center_of_gravity
Function 'compute_center_of_gravity' executed in 0.0186 seconds
Calling compute_center_of_vo

c:\Users\nc298\AppData\Local\anaconda3\envs\voxelizer\lib\site-packages\pandas\core\frame.py:10866: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\nc298\AppData\Local\anaconda3\envs\voxelizer\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\nc298\AppData\Local\anaconda3\envs\voxelizer\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\Users\nc298\AppData\Local\anaconda3\envs\voxelizer\lib\site-packages\pandas\core\frame.py:10866: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\nc298\AppData\Local\anaconda3\envs\voxelizer\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\nc298\AppData\Local\anaconda3\envs\voxelizer\li

Apply the filter to the reduced data

In [8]:
vasp.filter_attributes(
    filter_attribute=vox_filter["filter_attribute"],
    filter_value=vox_filter["filter_value"],
    min_max_eq=vox_filter["min_max_eq"]
    )

Calling filter_attributes
Function 'filter_attributes' executed in 0.0000 seconds


Save result as a point cloud with new attributes

In [9]:
data_handler.df = vasp.df
data_handler.save_as_las(
    outfile=outfile
    )

Calling save_as_las
Error adding dimension 'big_int_index': no field of name big_int_index
Adding new dim 'big_int_index'
Error adding dimension 'hash_index': no field of name hash_index
Adding new dim 'hash_index'
Error adding dimension 'point_density': no field of name point_density
Adding new dim 'point_density'
Error adding dimension 'cov_xx': no field of name cov_xx
Adding new dim 'cov_xx'
Error adding dimension 'cov_xy': no field of name cov_xy
Adding new dim 'cov_xy'
Error adding dimension 'cov_xz': no field of name cov_xz
Adding new dim 'cov_xz'
Error adding dimension 'cov_yx': no field of name cov_yx
Adding new dim 'cov_yx'
Error adding dimension 'cov_yy': no field of name cov_yy
Adding new dim 'cov_yy'
Error adding dimension 'cov_yz': no field of name cov_yz
Adding new dim 'cov_yz'
Error adding dimension 'cov_zx': no field of name cov_zx
Adding new dim 'cov_zx'
Error adding dimension 'cov_zy': no field of name cov_zy
Adding new dim 'cov_zy'
Error adding dimension 'cov_zz': no

Save result as .ply file containing a mesh of cubes/voxels.

In [10]:
data_handler.save_as_ply(
    outfile=outfile_ply,
    voxel_size=vasp.voxel_size,
    shift_to_center=False
    )

Calling save_as_ply
Calling _generate_mesh_data
Function '_generate_mesh_data' executed in 0.0064 seconds
Function 'save_as_ply' executed in 0.0087 seconds
